In [12]:
import os
from surprise import SVD
from surprise import dump
from surprise import Reader
from surprise import Dataset


import pandas as pd # pandas is a data manipulation library
import numpy as np #provides numerical arrays and functions to manipulate the arrays efficiently
import random
import re
import matplotlib.pyplot as plt # data visualization library
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS #used to generate world cloud
import networkx as nx

In [22]:
#rating 데이터 불러오기
top200_movie_rating = pd.read_csv('top200_movie_rating')
top200_movie = pd.read_csv('top200_movie')

In [23]:
top200_movie

,Unnamed: 0,title,movieId,genres,movie_year,title_only
0,0,The Martian (2015),134130,Adventure|Drama|Sci-Fi,2015.0,The Martian
1,1,Deadpool (2016),122904,Action|Adventure|Comedy|Sci-Fi,2016.0,Deadpool
2,2,Mad Max: Fury Road (2015),122882,Action|Adventure|Sci-Fi|Thriller,2015.0,Mad Max: Fury Road
3,3,Inside Out (2015),134853,Adventure|Animation|Children|Comedy|Drama|Fantasy,2015.0,Inside Out
4,4,Ex Machina (2015),115713,Drama|Sci-Fi|Thriller,2015.0,Ex Machina
5,5,Star Wars: Episode VII - The Force Awakens (2015),122886,Action|Adventure|Fantasy|Sci-Fi|IMAX,2015.0,Star Wars: Episode VII - The Force Awakens
6,6,Arrival (2016),164179,Sci-Fi,2016.0,Arrival
7,7,Kingsman: The Secret Service (2015),119145,Action|Adventure|Comedy|Crime,2015.0,Kingsman: The Secret Service
8,8,Zootopia (2016),152081,Action|Adventure|Animation|Children|Comedy,2016.0,Zootopia
9,9,"Big Short, The (2015)",148626,Drama,2015.0,"Big Short, The"


In [3]:
#max 컬럼수 조정
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

In [4]:
top200_movie_rating.head(200)

,Unnamed: 0,title,movieId,genres,movie_year,title_only,userId,rating
0,0,The Martian (2015),134130,Adventure|Drama|Sci-Fi,2015.0,The Martian,7439,4.0
1,1,Zootopia (2016),152081,Action|Adventure|Animation|Children|Comedy,2016.0,Zootopia,7439,4.5
2,2,Spotlight (2015),142488,Thriller,2015.0,Spotlight,7439,5.0
3,3,Passengers (2016),166635,Adventure|Drama|Romance|Sci-Fi,2016.0,Passengers,7439,3.5
4,4,Lady Bird (2017),177615,Comedy,2017.0,Lady Bird,7439,4.0
5,5,Maze Runner: Scorch Trials (2015),117895,Action|Thriller,2015.0,Maze Runner: Scorch Trials,7439,2.5
6,6,Kingsman: The Golden Circle (2017),176101,Action|Adventure|Comedy,2017.0,Kingsman: The Golden Circle,7439,4.0
7,7,Lion (2016),165551,Drama,2016.0,Lion,7439,4.5
8,8,The Good Dinosaur (2015),136016,Adventure|Animation|Children|Comedy|Fantasy,2015.0,The Good Dinosaur,7439,4.0
9,9,Bright (2017),182823,Action|Crime|Fantasy,2017.0,Bright,7439,3.5


In [5]:
reader = Reader(rating_scale=(0.5, 5))

In [6]:
col = ['userId', 'title_only', 'rating']
top200_movie_rating = top200_movie_rating[col]

In [13]:
top200_movie_rating

,userId,title_only,rating
0,7439,The Martian,4.0
1,7439,Zootopia,4.5
2,7439,Spotlight,5.0
3,7439,Passengers,3.5
4,7439,Lady Bird,4.0
...,...,...,...
12585,6553,The Witch,4.5
12586,6553,War Dogs,2.5
12587,6553,War for the Planet of the Apes,1.5
12588,6553,The Conjuring 2,0.5


In [14]:
data = Dataset.load_from_df(top200_movie_rating, reader)

In [15]:
data

In [16]:
trainset = data.build_full_trainset()

In [17]:
trainset

In [18]:
algo = SVD()
algo.fit(trainset)

In [19]:
pred = algo.predict(6553, 'The Conjuring 2')

In [20]:
pred.est

2.4833842770340198

In [86]:
userId = 153348

In [87]:
seen_movies = top200_movie_rating[top200_movie_rating['userId']==userId]['title_only'].tolist()

In [88]:
seen_movies

['The Martian',
 'Avengers: Infinity War - Part I',
 'Sicario',
 'Spectre',
 'Passengers',
 'Now You See Me 2',
 'Insurgent',
 'Game Night',
 'Jurassic World: Fallen Kingdom',
 'Atomic Blonde',
 'The Post']

In [89]:
total_movies = top200_movie['title_only'].tolist()

In [90]:
total_movies

['The Martian',
 'Deadpool',
 'Mad Max: Fury Road',
 'Inside Out',
 'Ex Machina',
 'Star Wars: Episode VII - The Force Awakens',
 'Arrival',
 'Kingsman: The Secret Service',
 'Zootopia',
 'Big Short, The',
 'The Revenant',
 'Avengers: Age of Ultron',
 'Rogue One: A Star Wars Story',
 'Logan',
 'Guardians of the Galaxy 2',
 'Doctor Strange',
 'Avengers: Infinity War - Part I',
 'Ant-Man',
 'The Hateful Eight',
 'Captain America: Civil War',
 'Jurassic World',
 'Thor: Ragnarok',
 'Spotlight',
 'Get Out',
 'Blade Runner 2049',
 'The Hunger Games: Mockingjay - Part 2',
 'Mission: Impossible - Rogue Nation',
 'Dunkirk',
 'Sicario',
 'Deadpool 2',
 'Fantastic Beasts and Where to Find Them',
 'Untitled Spider-Man Reboot',
 'Wonder Woman',
 'Room',
 '10 Cloverfield Lane',
 'Baby Driver',
 'Moana',
 'Bridge of Spies',
 'Coco',
 'La La Land',
 'Three Billboards Outside Ebbing, Missouri',
 'The Man from U.N.C.L.E.',
 'Star Wars: The Last Jedi',
 'Spectre',
 'Black Panther',
 'The Nice Guys',
 'Fi

In [91]:
unseen_movies = [ movie for movie in total_movies if movie not in seen_movies]

In [92]:
unseen_movies

['Deadpool',
 'Mad Max: Fury Road',
 'Inside Out',
 'Ex Machina',
 'Star Wars: Episode VII - The Force Awakens',
 'Arrival',
 'Kingsman: The Secret Service',
 'Zootopia',
 'Big Short, The',
 'The Revenant',
 'Avengers: Age of Ultron',
 'Rogue One: A Star Wars Story',
 'Logan',
 'Guardians of the Galaxy 2',
 'Doctor Strange',
 'Ant-Man',
 'The Hateful Eight',
 'Captain America: Civil War',
 'Jurassic World',
 'Thor: Ragnarok',
 'Spotlight',
 'Get Out',
 'Blade Runner 2049',
 'The Hunger Games: Mockingjay - Part 2',
 'Mission: Impossible - Rogue Nation',
 'Dunkirk',
 'Deadpool 2',
 'Fantastic Beasts and Where to Find Them',
 'Untitled Spider-Man Reboot',
 'Wonder Woman',
 'Room',
 '10 Cloverfield Lane',
 'Baby Driver',
 'Moana',
 'Bridge of Spies',
 'Coco',
 'La La Land',
 'Three Billboards Outside Ebbing, Missouri',
 'The Man from U.N.C.L.E.',
 'Star Wars: The Last Jedi',
 'Black Panther',
 'The Nice Guys',
 'Finding Dory',
 'Spider-Man: Into the Spider-Verse',
 'The Intern',
 'Incredib

In [93]:
predictions =[]

In [94]:
for movieName in unseen_movies:
    predictions.append(algo.predict(str(userId), str(movieName)))

In [95]:
def sortkey_est(pred):
        return pred.est

In [96]:
predictions

[Prediction(uid='153348', iid='Deadpool', r_ui=None, est=3.8491398120178477, details={'was_impossible': False}),
 Prediction(uid='153348', iid='Mad Max: Fury Road', r_ui=None, est=3.9327727637742367, details={'was_impossible': False}),
 Prediction(uid='153348', iid='Inside Out', r_ui=None, est=3.8339664425556137, details={'was_impossible': False}),
 Prediction(uid='153348', iid='Ex Machina', r_ui=None, est=3.8461287256460595, details={'was_impossible': False}),
 Prediction(uid='153348', iid='Star Wars: Episode VII - The Force Awakens', r_ui=None, est=3.74882071633117, details={'was_impossible': False}),
 Prediction(uid='153348', iid='Arrival', r_ui=None, est=3.945664947895167, details={'was_impossible': False}),
 Prediction(uid='153348', iid='Kingsman: The Secret Service', r_ui=None, est=3.7096621403696624, details={'was_impossible': False}),
 Prediction(uid='153348', iid='Zootopia', r_ui=None, est=3.8978788805975833, details={'was_impossible': False}),
 Prediction(uid='153348', iid='B

In [97]:
predictions.sort(key=sortkey_est, reverse=True)

In [98]:
top_predictions = predictions[:20]

In [99]:
top_predictions

[Prediction(uid='153348', iid='Planet Earth II', r_ui=None, est=4.216464057531762, details={'was_impossible': False}),
 Prediction(uid='153348', iid='Spider-Man: Into the Spider-Verse', r_ui=None, est=4.089117941234949, details={'was_impossible': False}),
 Prediction(uid='153348', iid='Your Name.', r_ui=None, est=4.056220999513121, details={'was_impossible': False}),
 Prediction(uid='153348', iid='Three Billboards Outside Ebbing, Missouri', r_ui=None, est=3.9612524140141834, details={'was_impossible': False}),
 Prediction(uid='153348', iid='Piper', r_ui=None, est=3.9495255727225524, details={'was_impossible': False}),
 Prediction(uid='153348', iid='Arrival', r_ui=None, est=3.945664947895167, details={'was_impossible': False}),
 Prediction(uid='153348', iid='Big Short, The', r_ui=None, est=3.9444901690294443, details={'was_impossible': False}),
 Prediction(uid='153348', iid='Spotlight', r_ui=None, est=3.94190265740182, details={'was_impossible': False}),
 Prediction(uid='153348', iid='A

In [100]:
for pred in top_predictions:
    movie_name = pred.iid
    movie_predicted_rating = pred.est
    
    print(movie_name, movie_predicted_rating)

Planet Earth II 4.216464057531762
Spider-Man: Into the Spider-Verse 4.089117941234949
Your Name. 4.056220999513121
Three Billboards Outside Ebbing, Missouri 3.9612524140141834
Piper 3.9495255727225524
Arrival 3.945664947895167
Big Short, The 3.9444901690294443
Spotlight 3.94190265740182
Avengers: Infinity War - Part II 3.938702623696058
Mad Max: Fury Road 3.9327727637742367
Get Out 3.9311799825023166
Manchester by the Sea 3.9287143531694877
Louis C.K.: Live at The Comedy Store 3.9238792328187557
Coco 3.9084024735444096
Room 3.8992985593266782
Zootopia 3.8978788805975833
The Handmaiden 3.8713824519801223
Kubo and the Two Strings 3.864519839537844
Hunt for the Wilderpeople 3.8623233744618184
Sing Street 3.8597947181781307
